In [1]:
import requests
import time
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json

In [23]:
def getIndeksLink(links, page, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "http://www.tribunnews.com/index-news?date="+date+"&page="+str(page)
    print(url)
     #scarp with selenium
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessary.
    options.add_argument('--disable-extensions')

    driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
    driver.get(url)
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(driver.page_source, "html5lib")
    contentDiv = soup.find('div', class_="pt10 pb10")
    indeks = contentDiv.findAll('li', class_='ptb15')
    for post in indeks:
        link = [post.find('a', href=True)['href'], ""]
        links.append(link)
        
    el_page = soup.find('div', class_="paging")
    if el_page:
        check_link = el_page.findAll('a')[-1]
        if "id" in check_link:
            max_page = page
        else:
            max_page = int(check_link['href'].replace('\n', '').strip(' ')[58:])
    
        if page < max_page:
            links = getIndeksLink(links, page+1, date)
        
    return links

In [24]:
def getDetailBerita(links):
    """
    Mengambil seluruh element dari halaman berita
    """
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #extract subcategory from meta
        sub = html.unescape(soup.find('meta', attrs={'property': 'article:section'})['content'])

        articles['subcategory'] = sub
        
        articles['id'] = int(soup.find("link", attrs={'rel':'shortlink'})['href'].replace("https://www.seva.id/otomotif/?p=", "").strip(' \t\n\r'))
        #category
        articles['category'] = link[1]
        articles['url'] = url
        
        article = soup.find('div', class_="news-content border-list")
        
        #extract date
        #2018-07-27T15:18:00+00:00
        pubdate = soup.find("meta", attrs={'property':'article:published_time'})['content']
        pubdate = pubdate[0:19].strip(' \t\n\r')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%dT%H:%M:%S"), '%Y-%m-%d %H:%M:%S')
        
        #extract author
        articles['author'] = soup.find("div", class_="col-md-8").find('div', class_='col-md-10').find('div', class_="details").text
        
        #extract title
        articles['title'] = article.find('div', class_="title").find('h1').text
        
        #source
        articles['source'] = 'seva'
        
        #extract comments count
        articles['comments'] = 0
        
        #extract tags
        tags = soup.findAll('meta', attrs={'property':'article:tag'})
        articles['tags'] = ','.join([x['content'] for x in tags])
        
        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']
        
        #extract detail
        detail = article.find('div', class_="content")
        
        
        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = html.unescape(content)
        print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [25]:
getIndeksLink([], 1)

page  1
http://www.tribunnews.com/index-news?date=2018-07-31&page=1
page  2
http://www.tribunnews.com/index-news?date=2018-07-31&page=2
page  3
http://www.tribunnews.com/index-news?date=2018-07-31&page=3
page  4
http://www.tribunnews.com/index-news?date=2018-07-31&page=4
page  5
http://www.tribunnews.com/index-news?date=2018-07-31&page=5
page  6
http://www.tribunnews.com/index-news?date=2018-07-31&page=6
page  7
http://www.tribunnews.com/index-news?date=2018-07-31&page=7
page  8
http://www.tribunnews.com/index-news?date=2018-07-31&page=8
page  9
http://www.tribunnews.com/index-news?date=2018-07-31&page=9


KeyboardInterrupt: 